balloon数据集基于coco数据集，对mask rcnn的finetune

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv-full

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git

In [ ]:
!pip install -r requirements/build.txt

In [ ]:
!pip install -v -e .

In [ ]:
# 上传处理好的数据集
!unzip /content/mmdetection/data/balloon.zip

In [44]:
# 新建configs/mask_rcnn50/  放入配置文件
!python tools/train.py /content/mmdetection/configs/mask_rcnn50/mask_rcnn_r50_fpn_2x_balloon.py

/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
2023-02-09 10:50:36,495 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.10 (default, Nov 14 2022, 12:59:47) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.2, V11.2.152
GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 1.13.1+cu116
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 ar

In [64]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import cv2
import numpy as np 
# 指定模型的配置文件和 checkpoint 文件路径
config_file = '/content/mmdetection/work_dirs/mask_rcnn_r50_fpn_2x_balloon/mask_rcnn_r50_fpn_2x_balloon.py'
checkpoint_file = '/content/mmdetection/work_dirs/mask_rcnn_r50_fpn_2x_balloon/latest.pth'

# 根据配置文件和 checkpoint 文件构建模型
model = init_detector(config_file, checkpoint_file, device='cuda:0')


# # 在一个新的窗口中将结果可视化
# model.show_result(img, result)
# # 或者将可视化结果保存为图片
# model.show_result(img, result, out_file='result.jpg')

# 测试视频并展示结果
video = mmcv.VideoReader('/content/mmdetection/data/balloon/test_video.mp4')
frame_id = 0
for frame in video:
    # bgr转灰度
    r = frame[:,:,-1]
    b = frame[:,:,0]
    g = frame[:,:,1]
    gray = (r**1.8*0.2446+g**1.8*0.6720+b**1.8*0.0833)**(1/1.8) # 转灰度
    gray =gray.astype(np.uint8) # 转uint8
    gray = np.expand_dims(gray, axis=2) # 升维
    gray = np.concatenate((gray,gray,gray),axis=2) # 拼接灰度三通道 

    # 推理
    result = inference_detector(model, frame)
    masks = result[-1][0] # [mask]

    #mask取并
    mask_sum = np.zeros(frame.shape[0:2],dtype=bool)
    for mask in masks:
        mask_sum = mask+mask_sum # 布尔或运算
    gray[mask_sum] = frame[mask_sum] # 色彩拼接
    # print(mask_sum)
    # print(gray)
    # cv2.imshow('gray',gray)
    # print('frame',frame.shape)
    cv2.imwrite('/content/mmdetection/colorframes/{}.jpg'.format("%06d"%frame_id), gray)
    frame_id += 1
    #print('frame {} has been transformed'.format(frame_id))
print('frame has been transformed')    

load checkpoint from local path: /content/mmdetection/work_dirs/mask_rcnn_r50_fpn_2x_balloon/latest.pth


/content/mmdetection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


frame has been transformed


In [65]:
# 图片转换为视频
mmcv.frames2video('/content/mmdetection/colorframes', '/content/mmdetection/colorframes/color_splash.mp4')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 250/250, 34.4 task/s, elapsed: 7s, ETA:     0s
